In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
# Load MovieLens dataset
def load_movielens_data():
    url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
    df = pd.read_csv(url, sep='\t', names=['user_id', 'item_id', 'rating', 'timestamp'])
    return df

In [3]:
# Split into control (A) and treatment (B) groups
def split_data(df):
    df['group'] = np.random.choice(['A', 'B'], size=len(df), p=[0.5, 0.5])
    return df


In [4]:
# Compute A/B test metrics
def ab_test_results(df):
    avg_rating_A = df[df['group'] == 'A']['rating'].mean()
    avg_rating_B = df[df['group'] == 'B']['rating'].mean()
    uplift = ((avg_rating_B - avg_rating_A) / avg_rating_A) * 100

    print(f"Average Rating in Control (A): {avg_rating_A:.2f}")
    print(f"Average Rating in Treatment (B): {avg_rating_B:.2f}")
    print(f"Percentage Uplift: {uplift:.2f}%")

In [5]:
# Multi-Armed Bandit (Epsilon-Greedy)
def multi_armed_bandit(df, epsilon=0.1, iterations=1000):
    movie_rewards = {}
    movie_counts = {}

    for i in range(iterations):
        if np.random.rand() < epsilon:
            selected_movie = np.random.choice(df['item_id'].unique())  # Explore
        else:
            if movie_rewards:
                selected_movie = max(movie_rewards, key=lambda k: movie_rewards[k] / movie_counts[k])  # Exploit
            else:
                selected_movie = np.random.choice(df['item_id'].unique())

        reward = df[df['item_id'] == selected_movie]['rating'].mean()

        if selected_movie in movie_rewards:
            movie_rewards[selected_movie] += reward
            movie_counts[selected_movie] += 1
        else:
            movie_rewards[selected_movie] = reward
            movie_counts[selected_movie] = 1

    best_movie = max(movie_rewards, key=lambda k: movie_rewards[k] / movie_counts[k])
    print(f"Best movie selected by Multi-Armed Bandit: {best_movie}")

In [6]:
# Run the experiment
df = load_movielens_data()
df = split_data(df)
ab_test_results(df)
multi_armed_bandit(df)

Average Rating in Control (A): 3.53
Average Rating in Treatment (B): 3.53
Percentage Uplift: -0.07%
Best movie selected by Multi-Armed Bandit: 127
